![CH3-ADS.png](./Media/CH3-ADS.png)

  



# <span style="color: rgb(204, 85, 0);">Gather Buffer Pool Configuration</span>

## Benefits of the buffer pool extension

The primary purpose of a SQL Server database is to store and retrieve data, so intensive disk I/O is a core characteristic of the Database Engine. Because disk I/O operations can consume many resources and take a relatively long time to finish, SQL Server focuses on making I/O highly efficient. The buffer pool serves as a primary memory allocation source of SQL Server.  Buffer management is a key component in achieving this efficiency. The buffer management component consists of two mechanisms: the buffer manager to access and update database pages, and the buffer pool, to reduce database file I/O.

Data and index pages are read from disk into the buffer pool and modified pages (also known as dirty pages) are written back to disk. Memory pressure on the server and database checkpoints cause hot (active) dirty pages in the buffer cache to be evicted from the cache and written to mechanical disks and then read back into the cache. These I/O operations are typically small random reads and writes on the order of 4 to 16 KB of data. Small random I/O patterns incur frequent seeks, competing for the mechanical disk arm, increasing I/O latency, and reducing aggregate I/O throughput of the system.

The typical approach to resolving these I/O bottlenecks is to add more RAM, or alternatively, added high-performance SAS spindles. While these options are helpful, they have significant drawbacks: RAM is more expensive than data storage drives and adding spindles increases capital expenditure in hardware acquisition and increases operational costs by increased power consumption and increased probability of component failure.

The buffer pool extension feature extends the buffer pool cache with nonvolatile storage (usually SSD). Because of this extension, the buffer pool can accommodate a larger database working set, which forces the paging of I/Os between RAM and the SSDs. This effectively offloads small random I/Os from mechanical disks to SSDs. Because of the lower latency and better random I/O performance of SSDs, the buffer pool extension significantly improves I/O throughput.

The following list describes the benefits of the buffer pool extension feature.

- Increased random I/O throughput
- Reduced I/O latency
- Increased transaction throughput
- Improved read performance with a larger hybrid buffer pool

## Concepts

The following terms are applicable to the buffer pool extension feature.

Solid-state drive (SSD) -- Solid-state drives store data in memory (RAM) in a persistent manner.

Buffer -- In SQL Server, A buffer is an 8-KB page in memory, the same size as a data or index page. Thus, the buffer cache is divided into 8-KB pages. A page remains in the buffer cache until the buffer manager needs the buffer area to read in more data. Data is written back to disk only if it is modified. These in-memory modified pages are known as dirty pages. A page is clean when it is equivalent to its database image on disk. Data in the buffer cache can be modified multiple times before being written back to disk.

Buffer pool -- Also called buffer cache. The buffer pool is a global resource shared by all databases for their cached data pages.  The maximum and minimum size of the buffer pool cache is determined during startup or when the instance of SQL server is dynamically reconfigured by using sp\_configure.  This size determines the maximum number of pages that can be cached in the buffer pool at any time in the running instance.  The maximum memory that can be committed by buffer pool extension can be limited by the other applications running on the machine in case those create significant memory pressure.

Checkpoint --A checkpoint creates a known good point from which the Database Engine can start applying changes contained in the transaction log during recovery after an unexpected shutdown or crash.  A checkpoint writes the dirty pages and transaction log information from memory to disk and, also, records information about the transaction log. 

### <span style="color: rgb(0, 204, 153)">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

In [ ]:
SET NOCOUNT ON;
SET ANSI_WARNINGS ON;
SET QUOTED_IDENTIFIER ON;

DECLARE @sqlmajorver int

SELECT @sqlmajorver = CONVERT(int, (@@microsoftversion / 0x1000000) & 0xff);

IF @sqlmajorver > 11
BEGIN
	SELECT 'Information' AS [Category], 'BP_Extension' AS [Information], 
		CASE WHEN state = 0 THEN 'BP_Extension_Disabled' 
			WHEN state = 1 THEN 'BP_Extension_is_Disabling'
			WHEN state = 3 THEN 'BP_Extension_is_Enabling'
			WHEN state = 5 THEN 'BP_Extension_Enabled'
		END AS state, 
		[path], current_size_in_kb
	FROM sys.dm_os_buffer_pool_extension_configuration
END
ELSE
BEGIN
	SELECT 'Information' AS [Category], 'BP_Extension' AS [Information], 'NA' AS state
END;